### Problem Statement ###

In the telecom industry, customers are able to choose from multiple service providers and actively switch from one operator to another. In this highly competitive market, the telecommunications industry experiences an average of 15-25% annual churn rate. Given the fact that it costs 5-10 times more to acquire a new customer than to retain an existing one, customer retention has now become even more important than customer acquisition.o reduce customer churn, telecom companies need to predict which customers are at high risk of churn.

**Churn Phases**
- In ‘good’ phase the customer is happy with the service and behaves as usual
- In ‘action’ phase The customer experience starts to sore in this phase
- In ‘churn’ phase the customer is said to have churned

#### Business Goal ####

In this project, you will analyse customer-level data of a leading telecom firm, build predictive models to identify customers at high risk of churn and identify the main indicators of churn.

#### Outcomes ####

- Predict churn only on high-value customers
- Predict usage-based definition to define churn


### Step 1: Data Exploration

In [ ]:
# Suppressing Warnings
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# Importing Modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 250)
pd.set_option('display.max_columns', None)
pd.set_option('float_format', '{:.2f}'.format)


plt.style.use('fivethirtyeight')

In [ ]:
telecom_df = pd.read_csv('telecom_churn_data.csv')

In [ ]:
# Let's see the head of our master dataset
telecom_df.head()

In [ ]:
# Let's check the dimensions of the dataframe
telecom_df.shape

In [ ]:
# let's look at the statistical aspects of the dataframe
telecom_df.describe(include='all')

In [ ]:
# Let's see the type of each column
telecom_df.info(verbose=True)

In [ ]:
## Below columns are breaking the convention. So we will rename them appropriately
vbc_cols = [col for col in telecom_df.columns if 'vbc' in col]
print(vbc_cols)

In [ ]:
telecom_df.rename(columns = {'jun_vbc_3g': 'vbc_3g_6', 'jul_vbc_3g': 'vbc_3g_7', 'aug_vbc_3g': 'vbc_3g_8', 'sep_vbc_3g': 'vbc_3g_9'}, inplace=True)

In [ ]:
#DataType Correction 
object_df = telecom_df.select_dtypes(include='object')
object_df.head()

#Looks like all are datetime object

In [ ]:
#convert object to date time

for col in object_df.columns:
    telecom_df[col] = pd.to_datetime(telecom_df[col])

In [ ]:
telecom_df.shape

We are now ready to cleanse the data and create a Manageable Data Set for further processing

## Step 2 : Data Cleansing

**Common Utility Functions**

In [ ]:
# Function which returns the columns with missing values > the cutoff percentage
# Argument: cutoff percentage between 1 - 100
def calculate_missing_values(data, cutoff):
    missing_percent= round(data.isna().sum() / len(data.index) * 100)
    print("{} features having more than {}% missing values:".format(len(missing_percent[missing_percent > cutoff]), cutoff))
    return missing_percent[missing_percent > cutoff]

#Function to handle missing values across months 
# Argument: Take list of column names without month number suffix 
def impute_zero_in_missing(data, columnList):
    for feature in [col + suffix for suffix in ['_6','_7','_8','_9'] for col in columnList]:
        if feature in data.columns:
            data[feature].fillna(0, inplace=True)
        

#Function to drop columns across months
def drop_columns(data,columnList):
    for feature in [col+suffix for suffix in ['_6','_7','_8','_9'] for col in columnList]:
        data.drop([feature],inplace=True, axis=1)

Drop columns which have only null values

In [ ]:
telecom_df[telecom_df.isnull().all(axis=1)]

##Looks like there are no columns with only null values

Drop all columns with 1 unique value 

In [ ]:
unique_cols = telecom_df.nunique()
unique_cols[unique_cols == 1]

In [ ]:
#Drop mobile_number as it is unique value identifying each record
telecom_df.drop('mobile_number',inplace=True, axis=1)

In [ ]:
telecom_df.drop(unique_cols[unique_cols == 1].index, inplace=True, axis=1)

In [ ]:
telecom_df.shape

Drop columns with missing values greater than 50% or impute the same

In [ ]:
calculate_missing_values(telecom_df, 50)

Impute Missing Value as 0 for all the **Recharge, Revenue, Night Pack, Fb User** Fields

In [ ]:
missingValueColumnList=['total_rech_data', 'max_rech_data', 'count_rech_2g', 'count_rech_3g', 'av_rech_amt_data'
                       , 'arpu_3g', 'arpu_2g', 'night_pck_user', 'fb_user']

In [ ]:
# Since the minimum value is 1, we are going to handle NA values by imputing with 0 
# which means we are assuming there were no recharges done by the customer
impute_zero_in_missing(telecom_df, missingValueColumnList)

In [ ]:
calculate_missing_values(telecom_df, 50)

We will drop all features which have more than 70% missing values

In [ ]:
drop_columns(telecom_df,['date_of_last_rech_data'])

In [ ]:
calculate_missing_values(telecom_df, 7)

In [ ]:
calculate_missing_values(telecom_df, 7)

All columns except date columns are for the month 9 i.e the Churn phase. 
This data will be eventually dropped when we tag churn/no churn, 
hence we will be skipping imputing these fields related to month 9

### Step 3: Data Preparation ###

**Filter in High Value Customers which are the target of our analysis**

* A high-value customers is defined as follows:

- Those who have recharged with an amount more than or equal to X, where X is greater than 70th percentile of the average recharge amount in the first two months (the good phase)

In [ ]:
# Create new column total recharge amount for data for aiding in finding High Value customer 

telecom_df['total_rech_amt_data_6'] = telecom_df.av_rech_amt_data_6 * telecom_df.total_rech_data_6
telecom_df['total_rech_amt_data_7'] = telecom_df.av_rech_amt_data_7 * telecom_df.total_rech_data_7
telecom_df['total_rech_amt_data_8'] = telecom_df.av_rech_amt_data_8 * telecom_df.total_rech_data_8

In [ ]:
#Create column for holding average of total recharge amount for good phase (the months June (6) and July (7))
# We add the total recharge amount of call and data and find the average across two months
telecom_df['total_avg_rech_amnt_Good_Phase'] = (telecom_df.total_rech_amt_6 + telecom_df.total_rech_amt_data_6 \
                                               + telecom_df.total_rech_amt_7+ telecom_df.total_rech_amt_data_7)/2

In [ ]:
# filter values greater than 70th percentile of total average recharge amount for good phase 
seventieth_percentile = telecom_df.total_avg_rech_amnt_Good_Phase.quantile(0.7)

telecom_df_high_val_cust = telecom_df[telecom_df.total_avg_rech_amnt_Good_Phase > seventieth_percentile]

In [ ]:
print("70th Percentile of Average Recharge amount in Good Phase (June and July month) is ", seventieth_percentile)

In [ ]:
telecom_df_high_val_cust.shape


**Define the Target Variable Churn based on the following criteria**

* A Churned Customer is defined as follows : 
* A Customer has churned (churn=1, else 0) if in the Ninth Month he/she has not made any calls (either incoming or outgoing) 
* AND have not used mobile internet even once. 



In [ ]:
#Add a new column "churn", values would be either 1 (churn) or 0 (non-churn)
telecom_df_high_val_cust['churn'] = \
        np.where(telecom_df_high_val_cust[['total_ic_mou_9','total_og_mou_9', \
                                           'vol_2g_mb_9', 'vol_3g_mb_9']].sum(axis=1) == 0, 1,0)

In [ ]:
# Find out the % of churn/non churn customers
churn_percentage = telecom_df_high_val_cust.churn.value_counts(normalize=True)
print(churn_percentage)
churn_percentage.plot.bar()
plt.show()

**Observation** : The churn percentage is around 8%. This indicates that there is a slight imbalance in the dataset which will need to be corrected in modelling

**Drop all data of the ninth Month as that is our Target Variable**


In [ ]:
churn_month_columns =  telecom_df_high_val_cust.columns[telecom_df_high_val_cust.columns.str.contains('_9')]

In [ ]:
churn_month_columns

In [ ]:
# drop all columns corresponding to the churn phase
telecom_df_high_val_cust.drop(churn_month_columns,axis=1,inplace=True)

In [ ]:
telecom_df_high_val_cust.shape

### Step 4 : EDA ### 

We will now perform EDA and try to get insights into the data. 
Based on the insights we could define our approach to training, remove outliers, remove highly correlated variables

**Drop Highly Correlated Columns as a PreRequisite to EDA**

In [ ]:
## Creating a copy to avoid regenerating master for each iterations
##Check point 1
training_df = telecom_df_high_val_cust.copy()

In [ ]:
#Find Highly correlated data and drop Highly Correlated Columns
cor = training_df.corr()
cor.loc[:,:] = np.tril(cor, k=-1)
plt.figure(figsize=(15,10))
sns.heatmap(cor, cmap='Greens', annot=False, )
plt.show()

Looks like there are strong multicollinearity issues, Lets drop data which is multi collinear

In [ ]:
# Create correlation matrix
corr_matrix = training_df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.85
to_drop = [column for column in upper.columns if any(upper[column] > 0.85)]
to_drop

In [ ]:
#training_df.drop(to_drop, axis =1, inplace=True)

In [ ]:
training_df.shape

In [ ]:
#Find Highly correlated data and drop Highly Correlated Columns
cor = training_df.corr()
cor.loc[:,:] = np.tril(cor, k=-1)
sns.heatmap(cor, cmap='Greens', annot=False)

In [ ]:
#Checkpoint 2 
# We can run from checkpoints rather than run the entire notebook for validation
telecom_df_high_val_cust = training_df.copy()

#### Univariate Analysis 

We will trend the distribution of all categorical variables and see if we can draw any meaningful insights

In [ ]:
# Distribution graphs (histogram/bar graph) of column data
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            print(type(columnDf.iloc[0]))
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()

In [ ]:
plotPerColumnDistribution(telecom_df_high_val_cust,10,5)

#### Bi variate Analysis 
We will draw trends of the all categorical variables data wrt to the Label, Churn and see if we can derive any meaningful insights

In [ ]:
# create box plot for  6th, 7th and 8th month
def create_box_plot(column):
    plt.figure(figsize=(15,10))
    df = telecom_df_high_val_cust
    plt.subplot(2,3,1)
    sns.boxplot(data=df, y=column+"_6",x="churn", showfliers=False)
    plt.subplot(2,3,2)
    sns.boxplot(data=df, y=column+"_7",x="churn", showfliers=False)
    plt.subplot(2,3,3)
    sns.boxplot(data=df, y=column+"_8",x="churn", showfliers=False)
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()

In [ ]:
# create box plot for  6th, 7th and 8th month
def create_bar_plot(column):
    plt.figure(figsize=(15,10))
    df = telecom_df_high_val_cust
    plt.subplot(2,3,1)
    sns.barplot(data=df, y=column+"_6",x="churn")
    plt.subplot(2,3,2)
    sns.barplot(data=df, y=column+"_7",x="churn")
    plt.subplot(2,3,3)
    sns.barplot(data=df, y=column+"_8",x="churn")
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()

In [ ]:
def showbarlabel(graph, rotate=0):
    graph.set_xticklabels(graph.get_xticklabels(),rotation=rotate)
    for p in graph.patches:
        height = p.get_height()
        graph.text(p.get_x()+p.get_width()/2., height + 0.1,height ,ha="center")

In [ ]:
def plot_bar(by,rotate=0):
    df = telecom_df_high_val_cust
    graph = df[by].value_counts(sort=True).plot(kind='bar')
    showbarlabel(graph,rotate)

In [ ]:
# all Recharge Amount related column list
recharge_amnt_columns = [col for col in telecom_df_high_val_cust if 'rech_amt' in col.lower()]
recharge_amnt_columns

In [ ]:
100*telecom_df_high_val_cust[recharge_amnt_columns].isnull().sum()/len(telecom_df_high_val_cust.index)
# We don't have any missing values here

In [ ]:
# Ploting for Total recharge amount :
create_box_plot('total_rech_amt')

Total Recharge amount drops in month 8 indicating Churn 

In [ ]:
# Ploting for maximum recharge amount :
create_box_plot('max_rech_amt')

We can see that there is a huge drop in maximum recharge amount for churned customers in the 8th month i.e action phase


In [ ]:
create_box_plot('av_rech_amt_data')

Average Recharge amount drops in month 8 indicating Churn 

In [ ]:
# Ploting for total recharge amount data :
create_box_plot('total_rech_amt_data')


We can see a drop in the total recharge for data for churned customers in the 8th Month i.e Action Phase

In [ ]:
# all Recharge Number related column list
recharge_num_columns = [col for col in telecom_df_high_val_cust if 'rech_num' in col.lower()]
recharge_num_columns

In [ ]:
100*telecom_df_high_val_cust[recharge_num_columns].isnull().sum()/len(telecom_df_high_val_cust.index)
# We don't have any missing values here

In [ ]:
# Ploting for total recharge number:
create_box_plot('total_rech_num')

We can see that there is a huge drop in total recharge number for churned customers in the 8th month i.e action phase

In [ ]:
# all Recharge data related column list
recharge_data_columns = [col for col in telecom_df_high_val_cust if 'rech_data' in col.lower()]
recharge_data_columns

In [ ]:
100*telecom_df_high_val_cust[recharge_data_columns].isnull().sum()/len(telecom_df_high_val_cust.index)
# We don't have any missing values here

In [ ]:
# Ploting for total recharge data:
create_box_plot('total_rech_data')

Again we can see that there is a huge drop in total recharge amount data for churned customers in the 8th month i.e action phase

In [ ]:
# Ploting for max recharge for data:
create_box_plot('max_rech_data')

There is a huge drop in max recharge amount data for churned customers in the 8th month i.e action phase 

In [ ]:
# Ploting for Last  day recharge amount  :
create_box_plot('last_day_rch_amt')

 We are seeing a huge drop in recharge amount for churned customers in the 8th month i.e Action phase 

In [ ]:
all_2g_3g_columns =   telecom_df_high_val_cust.columns[telecom_df_high_val_cust.columns.str.contains('2g|3g')]
                                                                     
all_2g_3g_columns

In [ ]:
100*telecom_df_high_val_cust[all_2g_3g_columns].isnull().sum()/len(telecom_df_high_val_cust.index)
# We don't have any missing values here

In [ ]:
# Ploting for volume of 2G and 3G usage columns:
create_box_plot('vol_2g_mb')
create_box_plot('vol_3g_mb')

**Observation** 
We see 2g and 3g usage for churned customers drops in the 8th month i.e Action phase.

However in general we see the usage is low for churned customer across months. 

In [ ]:
# Ploting for count of 2G and 3G recharge columns:
create_bar_plot('count_rech_2g')
create_bar_plot('count_rech_3g')

**Observation** 
We see 2g and 3g recharge counts for churned customers drops in the 8th month i.e Action phase.



In [ ]:
# Ploting for arpu of 2G and 3G usage columns:
create_box_plot('arpu_2g')
create_box_plot('arpu_3g')

In [ ]:
# Ploting bar plot for arpu of 2G and 3G usage columns:
create_bar_plot('arpu_2g')
create_bar_plot('arpu_3g')

**Observation** 
We see 2g and 3g arpu for churned customers drops in the 8th month i.e Action phase.



In [ ]:
# Ploting for monthly subcription of 2G and 3G usage columns:
create_box_plot('monthly_2g')
create_box_plot('monthly_3g')

In [ ]:
# Plotting a bar plot as box plot doesn't show any pattern
create_bar_plot('monthly_2g')
create_bar_plot('monthly_3g')

**Observation** 
We see 2g and 3g monthly subscription for churned customers drops in the 8th month i.e Action phase.



In [ ]:
# Plotting for small duration subscription of 2g and 3g data
# Plotting a bar plot as box plot doesn't show any pattern
create_bar_plot('sachet_2g')
create_bar_plot('sachet_3g')

**Observation** 
We see 2g and 3g small duration subscription for churned customers drops in the 8th month i.e Action phase.





In [ ]:
#Plotting volume based 3g usage
create_bar_plot('vbc_3g')

Volume based 3G usage is much lower for Churned customers and also there is a drop in vbc in 8th month

In [ ]:
#Getting the  day  columns
day_columns = [col for col in telecom_df_high_val_cust if 'day' in col.lower()]
day_columns

In [ ]:
100*telecom_df_high_val_cust[day_columns].isnull().sum()/len(telecom_df_high_val_cust.index)
# We don't have any missing values here

In [ ]:
create_box_plot('last_day_rch_amt')

Huge drop in 8th month for last day recharge amount indicating  churn

In [ ]:
# all Date column list
date_columns = [col for col in telecom_df_high_val_cust if 'date' in col.lower()]
date_columns

In [ ]:
100*telecom_df_high_val_cust[date_columns].isnull().sum()/len(telecom_df_high_val_cust.index)


The missing value indicates that recharge date and the recharge value are missing together which means the customer didn't recharge for that month

In [ ]:
telecom_df_high_val_cust[telecom_df_high_val_cust.date_of_last_rech_6.isnull()][['total_rech_data_6','date_of_last_rech_6']].head()

In [ ]:
#Plot for ARPU 
create_box_plot('arpu')

**Observation** 
We see the ARPU for churned customers drops in the 8th month i.e Action phase.



In [ ]:
# all Minutes of Usage column list
mou_columns = [col for col in telecom_df_high_val_cust if 'mou' in col.lower()]
mou_columns

In [ ]:
missing_mou_columns = 100*telecom_df_high_val_cust[mou_columns].isnull().sum()/len(telecom_df_high_val_cust.index)
missing_mou_columns

In [ ]:
#Using set to have unique columns at the end after stripping month suffix
missing_mou_col_without_suffix = set()

for colname  in missing_mou_columns.index:
    missing_mou_col_without_suffix.add(str(colname)[:-2])

In [ ]:
missing_mou_col_without_suffix

We can fill the missing values with zero as it would be that the customer didn't use these services at all

In [ ]:
impute_zero_in_missing(telecom_df_high_val_cust, missing_mou_col_without_suffix)
100*telecom_df_high_val_cust[mou_columns].isnull().sum()/len(telecom_df_high_val_cust.index)

In [ ]:
#Plot for Onnet Minutes of Usage
create_box_plot('onnet_mou')

The calls on service provider network drops in month 8 indicates Churn

In [ ]:
#Plot for Offnet Minutes of Usage
create_box_plot('offnet_mou')

The calls to different network drops in month 8 indicates Churn